<a href="https://colab.research.google.com/github/Hehs33/TareasAprendizaje_PUCE/blob/main/2-Ejercicios/Optimizacion_Hiperparametros_TomasHidrobo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Tomas Hidrobo &bull; 2026-01
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [ ]:
# Paquetes necesarios
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier

import joblib

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [ ]:
df = pd.read_csv("/content/premier_league_cleaned.csv")
print("Dimensiones (filas, columnas):", df.shape)

print("\nPrimeras 10 filas:")
display(df.head(10))

print("\nDescripción (variables numéricas)")
display(df.describe().T)

print("\nDescripción (variables categóricas)")
display(df.describe(include="object").T)

Dimensiones (filas, columnas): (1049, 15)

Primeras 10 filas:


,Matchday,Date,Time,Home Team,homeScore,homeXG,awayScore,awayXG,Away Team,Attendance,Referee,Stadium,Result,*Additional Stats,Attendance_cap
0,1,8/5/2022,20:00:00,Crystal Palace,0,1.2,2,1.0,Arsenal,25286.0,Anthony Taylor,Selhurst Park,A,https://fbref.com//en/matches/e62f6e78/Crystal...,25286.0
1,1,8/6/2022,12:30:00,Fulham,2,1.2,2,1.2,Liverpool,22207.0,Andy Madley,Craven Cottage,D,https://fbref.com//en/matches/6713c1dc/Fulham-...,22508.5
2,1,8/6/2022,15:00:00,Tottenham,4,1.5,1,0.5,Southampton,61732.0,Andre Marriner,Tottenham Hotspur Stadium,H,https://fbref.com//en/matches/09d8a999/Tottenh...,61732.0
3,1,8/6/2022,15:00:00,Newcastle Utd,2,1.7,0,0.3,Nott'ham Forest,52245.0,Simon Hooper,St James' Park,H,https://fbref.com//en/matches/1ac96eb4/Newcast...,52245.0
4,1,8/6/2022,15:00:00,Leeds United,2,0.8,1,1.3,Wolves,36347.0,Robert Jones,Elland Road,H,https://fbref.com//en/matches/82702941/Leeds-U...,36347.0
5,1,8/6/2022,15:00:00,Bournemouth,2,0.6,0,0.7,Aston Villa,11013.0,Peter Bankes,Vitality Stadium,H,https://fbref.com//en/matches/877e3193/Bournem...,11013.0
6,1,8/6/2022,17:30:00,Everton,0,0.7,1,1.5,Chelsea,39254.0,Craig Pawson,Goodison Park,A,https://fbref.com//en/matches/3a917cee/Everton...,39254.0
7,1,8/7/2022,14:00:00,Leicester City,2,0.6,2,0.8,Brentford,31794.0,Jarred Gillett,King Power Stadium,D,https://fbref.com//en/matches/3249ba27/Leicest...,31794.0
8,1,8/7/2022,14:00:00,Manchester Utd,1,1.4,2,1.5,Brighton,73711.0,Paul Tierney,Old Trafford,A,https://fbref.com//en/matches/8251694e/Manches...,73711.0
9,1,8/7/2022,16:30:00,West Ham,0,0.5,2,2.2,Manchester City,62443.0,Michael Oliver,London Stadium,A,https://fbref.com//en/matches/ece62baf/West-Ha...,62443.0



Descripción (variables numéricas)


,count,mean,std,min,25%,50%,75%,max
Matchday,1049.0,18.246902,10.512853,1.0,9.0,18.0,27.0,38.0
homeScore,1049.0,1.660629,1.364484,0.0,1.0,1.0,2.0,9.0
homeXG,1049.0,1.626787,0.878962,0.0,1.0,1.5,2.2,7.0
awayScore,1049.0,1.383222,1.237897,0.0,0.0,1.0,2.0,8.0
awayXG,1049.0,1.316206,0.781492,0.0,0.8,1.2,1.7,5.6
Attendance,1049.0,39713.988084,17042.474631,9972.0,28997.0,38198.0,53306.0,75546.0
Attendance_cap,1049.0,39687.196735,17035.829814,9972.0,29116.0,36955.0,53306.0,74081.5



Descripción (variables categóricas)


,count,unique,top,freq
Date,1049,317,5/19/2024,10
Time,1049,21,15:00:00,384
Home Team,1049,24,Crystal Palace,53
Away Team,1049,24,Arsenal,53
Referee,1049,29,Anthony Taylor,82
Stadium,1049,27,Selhurst Park,53
Result,1049,3,H,473
*Additional Stats,1049,1049,https://fbref.com//en/matches/7bab156e/Leicest...,1


In [ ]:
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y", errors="coerce")
df["Matchday"] = pd.to_numeric(df["Matchday"], errors="coerce")

inicio_temp = pd.Timestamp("2024-08-16")

df = df[df["Date"] >= inicio_temp].copy()
df = df.sort_values(["Date", "Matchday"]).reset_index(drop=True)

print(df[["Date","Matchday"]].head(10))
print(df[["Date","Matchday"]].tail(10))

        Date  Matchday
0 2024-08-16         1
1 2024-08-17         1
2 2024-08-17         1
3 2024-08-17         1
4 2024-08-17         1
5 2024-08-17         1
6 2024-08-17         1
7 2024-08-18         1
8 2024-08-18         1
9 2024-08-19         1
          Date  Matchday
279 2025-03-09        28
280 2025-03-10        28
281 2025-03-15        29
282 2025-03-15        29
283 2025-03-15        29
284 2025-03-15        29
285 2025-03-15        29
286 2025-03-16        29
287 2025-03-16        29
288 2025-03-16        29


In [ ]:
local_a = df[["Date","Matchday","Home Team","Away Team","homeXG","awayXG","Result"]].copy()
local_a = local_a.rename(columns={
    "Home Team":"equipo", "Away Team":"oponente",
    "homeXG":"xG_afavor", "awayXG":"xG_encontra"
})
local_a["is_home"] = 1

visitante_a = df[["Date","Matchday","Home Team","Away Team","homeXG","awayXG","Result"]].copy()
visitante_a = visitante_a.rename(columns={
    "Away Team":"equipo", "Home Team":"oponente",
    "awayXG":"xG_afavor", "homeXG":"xG_encontra"
})
visitante_a["is_home"] = 0

equipo_temp = pd.concat([local_a, visitante_a], ignore_index=True)
equipo_temp = equipo_temp.sort_values(["equipo","Date","Matchday"]).reset_index(drop=True)


In [ ]:
equipo_temp["xG_afavor_temp"] = (
    equipo_temp.groupby("equipo")["xG_afavor"]
    .transform(lambda s: s.shift(1).expanding().mean())
)

equipo_temp["xG_encontra_temp"] = (
    equipo_temp.groupby("equipo")["xG_encontra"]
    .transform(lambda s: s.shift(1).expanding().mean())
)

equipo_temp["xG_dif_temp"] = equipo_temp["xG_afavor_temp"] - equipo_temp["xG_encontra_temp"]


In [ ]:
local_feats = equipo_temp[equipo_temp["is_home"] == 1][
    ["Date", "Matchday", "equipo", "xG_afavor_temp", "xG_encontra_temp", "xG_dif_temp"]
].rename(columns={
    "equipo": "Home Team",
    "xG_afavor_temp": "local_xG_afavor_temp",
    "xG_encontra_temp": "local_xG_encontra_temp",
    "xG_dif_temp": "local_xG_dif_temp"
})

visitante_feats = equipo_temp[equipo_temp["is_home"] == 0][
    ["Date", "Matchday", "equipo", "xG_afavor_temp", "xG_encontra_temp", "xG_dif_temp"]
].rename(columns={
    "equipo": "Away Team",
    "xG_afavor_temp": "visitante_xG_afavor_temp",
    "xG_encontra_temp": "visitante_xG_encontra_temp",
    "xG_dif_temp": "visitante_xG_dif_temp"
})

df_modelo = df.merge(local_feats, on=["Date", "Matchday", "Home Team"], how="left")
df_modelo = df_modelo.merge(visitante_feats, on=["Date", "Matchday", "Away Team"], how="left")

df_modelo["dif_xG_afavor_temp"] = df_modelo["local_xG_afavor_temp"] - df_modelo["visitante_xG_afavor_temp"]
df_modelo["dif_xG_dif_temp"] = df_modelo["local_xG_dif_temp"] - df_modelo["visitante_xG_dif_temp"]

df_modelo = df_modelo.dropna(subset=[
    "local_xG_afavor_temp", "visitante_xG_afavor_temp",
    "local_xG_encontra_temp", "visitante_xG_encontra_temp"
]).reset_index(drop=True)

print("Modelo listo:", df_modelo.shape)
df_modelo.head()

Modelo listo: (279, 23)


,Matchday,Date,Time,Home Team,homeScore,homeXG,awayScore,awayXG,Away Team,Attendance,...,*Additional Stats,Attendance_cap,local_xG_afavor_temp,local_xG_encontra_temp,local_xG_dif_temp,visitante_xG_afavor_temp,visitante_xG_encontra_temp,visitante_xG_dif_temp,dif_xG_afavor_temp,dif_xG_dif_temp
0,2,2024-08-24,12:30:00,Brighton,2,2.1,1,1.4,Manchester Utd,31537.0,...,https://fbref.com//en/matches/fc8ab8b2/Brighto...,31537.0,1.4,0.5,0.9,2.4,0.4,2.0,-1.0,-1.1
1,2,2024-08-24,15:00:00,Tottenham,4,2.4,0,1.0,Everton,61357.0,...,https://fbref.com//en/matches/1eef1717/Tottenh...,61357.0,1.2,1.0,0.2,0.5,1.4,-0.9,0.7,1.1
2,2,2024-08-24,15:00:00,Fulham,2,1.8,1,0.6,Leicester City,25401.0,...,https://fbref.com//en/matches/4d0079fb/Fulham-...,25401.0,0.4,2.4,-2.0,1.0,1.2,-0.2,-0.6,-1.8
3,2,2024-08-24,15:00:00,Crystal Palace,0,1.3,2,1.4,West Ham,25099.0,...,https://fbref.com//en/matches/540cfb68/Crystal...,25099.0,1.2,1.6,-0.4,2.3,2.0,0.3,-1.1,-0.7
4,2,2024-08-24,15:00:00,Manchester City,4,3.3,1,0.3,Ipswich Town,53147.0,...,https://fbref.com//en/matches/a24b7a43/Manches...,53147.0,0.8,1.0,-0.2,0.5,2.6,-2.1,0.3,1.9


Lo que hicimos en estas lineas del codigo es definir el xG de los equipos pero asegurandonos que solo sea esta temporada la cual inicia en el matchday 2 con fecha 2024-08-24. Asegurandonos de que el xG ya sea de visitante o local, se cuente y comience a promediarse desde esa fecha.
Considero que esto es importante ya que las temporadas pasadas no deberian afectar las actuales, dado que el equipo cambia (fichajes, cuerpo tecnico, etc) y en si el funcionamiento de cada equipo es diferente cada temporada.


### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [ ]:
y = df_modelo["Result"]

x = df_modelo[[
    "local_xG_afavor_temp",
    "local_xG_encontra_temp",
    "local_xG_dif_temp",
    "visitante_xG_afavor_temp",
    "visitante_xG_encontra_temp",
    "visitante_xG_dif_temp",
    "dif_xG_afavor_temp",
    "dif_xG_dif_temp",
    "Matchday"
]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,  random_state=99)

In [ ]:
# Crear y entrenar un arbol con ganancia de información
modelo_base = RandomForestClassifier(random_state=42)

# Parámetros del modelo
modelo_base.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

1. **max_depth**: qué tan “alto” dejas crecer cada árbol. Más alto = aprende más detalle (y puede sobreajustar); más bajo = más simple
2. **min_samples_leaf**: mínimo de datos que debe tener una “hoja” final. Si lo subes, el modelo no se pone tan quisquilloso con pocos casos.
3. **min_samples_split**: cuántos datos mínimos necesita un nodo para poder “partirse” en dos. Si lo subes, el árbol se divide menos.
4. **max_features**: cuántas variables mira el árbol cuando decide un corte. Si mira menos, hay más variedad entre árboles; si mira más, cada árbol suele ser más “fuerte” pero menos diverso.



Vamos a optimizar estos 4 hiperparametros ya que considero que son los que mas afectan al modelo.

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [ ]:
modelo_base = RandomForestClassifier(random_state=99)
k_fold = KFold(n_splits=5, shuffle=True, random_state=99)

parametros = {"max_depth": [None, 5, 10, 15, 20],"min_samples_leaf": [1, 2, 4, 6, 10],
    "min_samples_split": [2, 5, 10, 15, 20],"max_features": ["sqrt", "log2", 0.25, 0.5, 0.75],}

grid = GridSearchCV(estimator=modelo_base,param_grid=parametros,cv=k_fold,scoring="accuracy",
                    n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=99, shuffle=True),
             estimator=RandomForestClassifier(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': ['sqrt', 'log2', 0.25, 0.5, 0.75],
                         'min_samples_leaf': [1, 2, 4, 6, 10],
                         'min_samples_split': [2, 5, 10, 15, 20]},
             scoring='accuracy')

In [ ]:
print("Parámetros óptimos:", grid.best_params_)
print("Mejor score:", grid.best_score_)

Parámetros óptimos: {'max_depth': 10, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 5}
Mejor score: 0.565050505050505


### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [ ]:
parametros = {"max_depth": [None, 5, 10, 15, 20, 25],"min_samples_leaf": [1, 2, 4, 6, 8, 10],
    "min_samples_split": [2, 5, 7, 10, 13, 15],"max_features": ["sqrt", "log2", 0.2, 0.4, 0.6, 0.8],"n_estimators": [100, 250, 500, 750],}

rs = RandomizedSearchCV(estimator= modelo_base,param_distributions= parametros,n_iter=25,cv=k_fold,
    scoring="accuracy",random_state=99)
rs.fit(X_train, y_train)

print("Parámetros óptimos:", rs.best_params_)
print("Mejor score:", rs.best_score_)

Parámetros óptimos: {'n_estimators': 100, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': None}
Mejor score: 0.5474747474747474


### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [ ]:
modelo_random = RandomForestClassifier(
    random_state=99,
    n_estimators=100,
    max_depth=None,
    max_features="log2",
    min_samples_leaf=2,
    min_samples_split=7,
    n_jobs=-1
)

modelo_random.fit(X_train, y_train)
pred_random = modelo_random.predict(X_test)

acc_random = accuracy_score(y_test, pred_random)

modelo_grid = RandomForestClassifier(
    random_state=99,
    n_estimators=100,
    max_depth=10,
    max_features=0.5,
    min_samples_leaf=1,
    min_samples_split=5,
    n_jobs=-1
)

modelo_grid.fit(X_train, y_train)
pred_grid = modelo_grid.predict(X_test)

acc_grid = accuracy_score(y_test, pred_grid)

print("Precision con hiperparemtros de random:", acc_random)
print("Precision con hiperparametros de grid:", acc_grid)

Precision con hiperparemtros de random: 0.375
Precision con hiperparametros de grid: 0.39285714285714285


In [ ]:
modelo = RandomForestClassifier(
    random_state=99,
    n_estimators=500,
    max_depth=5,
    max_features=0.4,
    min_samples_leaf=8,
    min_samples_split=10,
    n_jobs=-1
)
modelo.fit(X_train, y_train)
print("Test accuracy:", modelo.score(X_test, y_test))

Test accuracy: 0.42857142857142855


In [ ]:
with open('modelo_premierleague.joblib', 'wb') as file:
    joblib.dump(modelo, file)

### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>